# Titanic Machine Learning from Disaster

In [1]:
import pandas as pd
import sklearn as sk

In [2]:
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')

Let's have a quick look at the data

In [3]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Cleaning

Looking for a way to fill NA in the *Embarked* column

In [5]:
train[['Embarked']].groupby(['Embarked']).size()

Embarked
C    168
Q     77
S    644
dtype: int64

We'll fill with 'S'

In [6]:
from sklearn.preprocessing import LabelEncoder  
le = LabelEncoder()

def clean_df(df):
    # Dropping the axis we'll not use
    df = df.drop(['Name', 'Ticket', 'Cabin', 'Fare'], axis=1)
    
    # Filling NAs
    df['Embarked'] = df[['Embarked']].fillna('S')
    df = df.fillna(df.mean())
    
    # Setting columns categories
    df['Sex'] = le.fit_transform(df['Sex'].astype('category'))
    df['Embarked'] = le.fit_transform(df['Embarked'].astype('category'))
    df['Age'] = df['Age'].astype('int64')
    
    return df

Cleaning the training and testing sets

In [7]:
train = clean_df(train)
test = clean_df(test)

## Creating the classifier

Splitting the data

In [8]:
X = train.drop('Survived', axis=1)
y = train['Survived']

Validating the model by CV

In [9]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [10]:
clf = svm.SVC(kernel='linear', C=1)
# clf = LogisticRegression()
scores = cross_val_score(clf, X, y, cv=10)

In [11]:
scores

array([0.81111111, 0.78888889, 0.7752809 , 0.84269663, 0.79775281,
       0.7752809 , 0.76404494, 0.74157303, 0.80898876, 0.76136364])

In [12]:
scores.mean()

0.7866981613891727

Fitting with the training set

In [13]:
clf.fit(X, y)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

Making predictions for the testing set

In [14]:
y_pred = clf.predict(test)

Creating a dataframe containing the results

In [15]:
df_res = pd.DataFrame(columns=['PassengerId', 'Survived'], data=test['PassengerId'])

In [16]:
df_res['Survived'] = y_pred

Exporting it to a csv file

In [17]:
df_res.to_csv('svm.csv', index=False)